<h1>Ensaio para Active Voxel</h1>

In [4]:
from vpython import *

ctr = vec( 150, 50, 150 )

# This is needed in Jupyter notebook and lab to make programs easily rerunnable
scene = canvas( center=ctr, background = color.gray( 0.8 ))
scene.autoscaling = False
scene.forward = vec( 0, 200, 0 )
scene.up = vec( 0, 0, 1 )
scene.camera.pos = vec( 150, -330, 150 )
scene.range = 50

<IPython.core.display.Javascript object>